In [ ]:
import numpy as np
from rail.core.data import TableHandle, DataStore, DATA_STORE
from rail.raruma import utility_functions as raruma_utils
import matplotlib.pyplot as plt
import qp

In [ ]:
DS = DATA_STORE()
DataStore.allow_overwrite = True

In [ ]:
ftag = "3951_v1"

In [ ]:
fl_data = DS.read_file('fl_data', TableHandle, f"/Users/echarles/pz/data/test/rubin_fl_{ftag}.hdf5")

In [ ]:
the_data = fl_data.data

In [ ]:
band_names = raruma_utils.make_band_names("{band}_cModelMag", 'ugri')
mags = raruma_utils.extract_data_to_2d_array(the_data, band_names)

In [ ]:
(dummy, mag_and_colors) = raruma_utils.prepare_data_total_mag_and_colors(the_data, "{band}_cModelMag", 'ugri')

In [ ]:
n_obs = np.isfinite(mags)

In [ ]:
n_bands = n_obs.sum(axis=1)

In [ ]:
(n_bands == 4).sum()

In [ ]:
mask = n_bands == 4

In [ ]:
algo = "knn"
qp_ens = qp.read(f"/Users/echarles/pz/projects/rubin_fl/data/gold_baseline/first_look_{ftag}/output_estimate_{algo}.hdf5")

In [ ]:
redshifts = np.squeeze(qp_ens.ancil['zmode'])

In [ ]:
_ = plt.scatter(the_data['ra'][mask], the_data['dec'][mask], c=redshifts[mask], cmap="rainbow", s=1)
_ = plt.colorbar(label='redshift')
_ = plt.xlabel("RA [deg]")
_ = plt.ylabel("DEC [deg]")

In [ ]:
_ = plt.hist(redshifts[mask], bins=np.linspace(0., 3., 61))
_ = plt.xlabel("redshift")
_ = plt.ylabel("Counts [per 0.05]")

In [ ]:
ids = the_data['id']

In [ ]:
import scipy.stats

In [ ]:
randoms = scipy.stats.truncnorm(a=-1, b=2, scale=1, loc=1).rvs(len(redshifts))

In [ ]:
output = np.where(mask, redshifts, randoms)

In [ ]:
out_data = dict(id=ids, redshifts=output)

In [ ]:
import tables_io

In [ ]:
tables_io.write(out_data, "fl_redshifts_3951.hdf5")

In [ ]:
d = tables_io.read("fl_redshifts_3951.hdf5")

In [ ]:
_ = plt.hist(d['redshifts'], bins=np.linspace(0., 3., 61))
_ = plt.xlabel("redshift")
_ = plt.ylabel("Counts [per 0.05]")

In [ ]:
qp_knn = qp.read(f"/Users/echarles/pz/projects/rubin_fl/data/gold_baseline/first_look_v1/output_estimate_knn.hdf5")
qp_fzb = qp.read(f"/Users/echarles/pz/projects/rubin_fl/data/gold_baseline/first_look_v1/output_estimate_fzboost.hdf5")
qp_tpz = qp.read(f"/Users/echarles/pz/projects/rubin_fl/data/gold_baseline/first_look_v1/output_estimate_tpz.hdf5")
qp_bpz = qp.read(f"/Users/echarles/pz/projects/rubin_fl/data/gold_baseline/first_look_v1/output_estimate_bpz.hdf5")
qp_dnf = qp.read(f"/Users/echarles/pz/projects/rubin_fl/data/gold_baseline/first_look_v1/output_estimate_dnf.hdf5")

qp_knns = qp.read(f"/Users/echarles/pz/projects/rubin_fl/data/gold_baseline/first_look_scarlet/output_estimate_knn.hdf5")
qp_fzbs = qp.read(f"/Users/echarles/pz/projects/rubin_fl/data/gold_baseline/first_look_scarlet/output_estimate_fzboost.hdf5")
qp_tpzs = qp.read(f"/Users/echarles/pz/projects/rubin_fl/data/gold_baseline/first_look_scarlet/output_estimate_tpz.hdf5")
qp_bpzs = qp.read(f"/Users/echarles/pz/projects/rubin_fl/data/gold_baseline/first_look_scarlet/output_estimate_bpz.hdf5")
qp_dnfs = qp.read(f"/Users/echarles/pz/projects/rubin_fl/data/gold_baseline/first_look_scarlet/output_estimate_dnf.hdf5")


In [ ]:
_ = plt.hist2d(np.squeeze(qp_knn.ancil['zmode'][mask]), np.squeeze(qp_knns.ancil['zmode'][mask]), bins=(60, 60), norm="log")
_ = plt.xlabel(r"$z_{\rm knn}$")
_ = plt.ylabel(r"$z_{\rm knn, scarlet}$")

In [ ]:
_ = plt.hist2d(np.squeeze(qp_dnf.ancil['zmode'][mask]), np.squeeze(qp_dnfs.ancil['zmode'][mask]), bins=(60, 60), norm="log")
_ = plt.xlabel(r"$z_{\rm dnf}$")
_ = plt.ylabel(r"$z_{\rm dnf, scarlet}$")